In [1]:
import pandas as pd
import numpy as np
import itertools
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [2]:
train_data = pd.read_csv('./data/sales_train.csv')
test_data = pd.read_csv('./data/test.csv')
item_data = pd.read_csv('./data/items.csv')

In [3]:
#将训练数据中的按天销量改为按月销量
train_data = train_data.drop(train_data[train_data.item_price < 0].index | train_data[train_data.item_price >= 100000].index)
train_data = train_data.drop(train_data[train_data.item_cnt_day < 0].index | train_data[train_data.item_cnt_day >= 1000].index)
train_data1 = pd.pivot_table(train_data, index=['shop_id','item_id','date_block_num'], values=['item_cnt_day'], aggfunc=[np.sum],fill_value=0).reset_index();
train_data1.columns = train_data1.columns.droplevel().map(str)
train_data1 = train_data1.reset_index(drop=True).rename_axis(None, axis=1)
train_data1.columns.values[0]='shop_id';
train_data1.columns.values[1]='item_id';
train_data1.columns.values[2]='month_id';
train_data1.columns.values[3]='item_cnt_month';
#由于题目的答案区间在0-20之间，所以我们限定销量在20以内
train_data1['item_cnt_month'] = (train_data1['item_cnt_month'].fillna(0).clip(0,20).astype(np.float16))

In [4]:
#数据增强
#如果商店该月没有销售该商品，则上面的数据中则没有该条数据。
#而实际上这时候该商店该月该商品的销售量应该等于0，这样更能体现商品、商店、销量的关系
matrix = []
cols = ['month_id','shop_id','item_id']
for i in range(34):
    sales = train_data[train_data.date_block_num==i]
    matrix.append(np.array(list(itertools.product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['month_id'] = matrix['month_id'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols, inplace=True)

In [5]:
#合并matrix和train_data1数据
train_data1 = pd.merge(matrix, train_data1, on=['shop_id', 'item_id', 'month_id'], how='left')
train_data1['item_cnt_month'] = (train_data1['item_cnt_month'].fillna(0).astype(np.float16))
train_data1['month_id1'] = train_data1['month_id'] % 12;

In [6]:
#获得每个商店物品组合的上月总销量
train_data2 = train_data1.copy()
train_data2.drop(['month_id1'], axis=1, inplace=True)
train_data2.columns.values[3] = 'item_cnt_month1'
train_data2['month_id'] = train_data2['month_id'] + 1

In [7]:
#获得每个物品上月的平均价格
item_price1 = pd.pivot_table(train_data, index=['item_id','date_block_num'], values=['item_price'], aggfunc=[np.mean], fill_value=0).reset_index();
item_price1.columns = item_price1.columns.droplevel().map(str)
item_price1 = item_price1.reset_index(drop=True).rename_axis(None, axis=1)
item_price1.columns.values[0]='item_id';
item_price1.columns.values[1]='month_id';
item_price1.columns.values[2]='item_mean_price1';
item_price1['month_id'] = item_price1['month_id'] + 1;

In [8]:
#获得每个商店上月的总销量
shop_cnt = pd.pivot_table(train_data1, index=['shop_id','month_id'], values=['item_cnt_month'], aggfunc=[np.sum], fill_value=0).reset_index();
shop_cnt.columns = shop_cnt.columns.droplevel().map(str)
shop_cnt = shop_cnt.reset_index(drop=True).rename_axis(None, axis=1)
shop_cnt.columns.values[0]='shop_id';
shop_cnt.columns.values[1]='month_id';
shop_cnt.columns.values[2]='shop_cnt_month1';
shop_cnt['month_id'] = shop_cnt['month_id'] + 1;

In [9]:
#获得每个物品上月的总销量
item_cnt = pd.pivot_table(train_data1, index=['item_id','month_id'], values=['item_cnt_month'], aggfunc=[np.sum], fill_value=0).reset_index();
item_cnt.columns = item_cnt.columns.droplevel().map(str)
item_cnt = item_cnt.reset_index(drop=True).rename_axis(None, axis=1)
item_cnt.columns.values[0]='item_id';
item_cnt.columns.values[1]='month_id';
item_cnt.columns.values[2]='item_total_month1';
item_cnt['month_id'] = item_cnt['month_id'] + 1;

In [10]:
#合并上述的五个特征到训练集中
combined_data = train_data1
combined_data = pd.merge(combined_data, train_data2, on = ['shop_id', 'item_id','month_id'], how = 'left')
combined_data['item_cnt_month1'].fillna(0, inplace=True)

combined_data = pd.merge(combined_data, item_price1, on = ['item_id','month_id'], how = 'left')
combined_data['item_mean_price1'].fillna(0, inplace=True)

combined_data = pd.merge(combined_data, shop_cnt, on = ['shop_id','month_id'], how = 'left')
combined_data['shop_cnt_month1'].fillna(0, inplace=True)

combined_data = pd.merge(combined_data, item_cnt, on = ['item_id', 'month_id'], how = 'left')
combined_data['item_total_month1'].fillna(0, inplace=True)

combined_data = pd.merge(combined_data, item_data, on = ['item_id'], how = 'left')
combined_data['item_category_id'].fillna(0, inplace=True)

In [11]:
print(test_data)

            ID  shop_id  item_id
0            0        5     5037
1            1        5     5320
2            2        5     5233
3            3        5     5232
4            4        5     5268
...        ...      ...      ...
214195  214195       45    18454
214196  214196       45    16188
214197  214197       45    15757
214198  214198       45    19648
214199  214199       45      969

[214200 rows x 3 columns]


In [12]:
#处理测试数据
test_data['month_id'] = 34
month_id = test_data.month_id
test_data.drop(['month_id'], axis=1, inplace=True)
test_data.insert(0, 'month_id', month_id)
test_data['month_id1'] = 10
test_data = pd.merge(test_data, train_data2, on = ['shop_id', 'item_id','month_id'], how = 'left')
test_data['item_cnt_month1'].fillna(0, inplace=True)
test_data = pd.merge(test_data, item_price1, on = ['item_id','month_id'], how = 'left')
test_data['item_mean_price1'].fillna(0, inplace=True)
test_data = pd.merge(test_data, shop_cnt, on = ['shop_id','month_id'], how = 'left')
test_data['shop_cnt_month1'].fillna(0, inplace=True)
test_data = pd.merge(test_data, item_cnt, on = ['item_id', 'month_id'], how = 'left')
test_data['item_total_month1'].fillna(0, inplace=True)
test_data = pd.merge(test_data, item_data, on = ['item_id'], how = 'left')
test_data['item_category_id'].fillna(0, inplace=True)

In [13]:
#删去物品名称\ID
combined_data.drop(['item_name'], axis=1, inplace=True)
test_data.drop('item_name', axis=1, inplace=True)
ID = test_data.ID
test_data.drop('ID', axis=1, inplace=True)

In [14]:
Train_data = combined_data.drop(['item_cnt_month'], axis=1)
Target_data = combined_data['item_cnt_month']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(Train_data, Target_data, random_state=42)

In [16]:
#模型训练
model = xgb.XGBRegressor(max_depth=5, 
                         learning_rate=0.1, 
                         n_estimators=128, 
                         min_child_weight=2
                        );
model.fit(X_train, 
          y_train,
          eval_metric='rmse',
          eval_set=[(X_test,y_test)]
         )

[0]	validation_0-rmse:1.20101
[1]	validation_0-rmse:1.16433
[2]	validation_0-rmse:1.13262
[3]	validation_0-rmse:1.10512
[4]	validation_0-rmse:1.08572
[5]	validation_0-rmse:1.06640
[6]	validation_0-rmse:1.05196
[7]	validation_0-rmse:1.03803
[8]	validation_0-rmse:1.02847
[9]	validation_0-rmse:1.01879
[10]	validation_0-rmse:1.01151
[11]	validation_0-rmse:1.00441
[12]	validation_0-rmse:0.99895
[13]	validation_0-rmse:0.99501
[14]	validation_0-rmse:0.99124
[15]	validation_0-rmse:0.98752
[16]	validation_0-rmse:0.98498
[17]	validation_0-rmse:0.98199
[18]	validation_0-rmse:0.97950
[19]	validation_0-rmse:0.97754
[20]	validation_0-rmse:0.97560
[21]	validation_0-rmse:0.97424
[22]	validation_0-rmse:0.97261
[23]	validation_0-rmse:0.97096
[24]	validation_0-rmse:0.96976
[25]	validation_0-rmse:0.96871
[26]	validation_0-rmse:0.96773
[27]	validation_0-rmse:0.96695
[28]	validation_0-rmse:0.96606
[29]	validation_0-rmse:0.96516
[30]	validation_0-rmse:0.96456
[31]	validation_0-rmse:0.96399
[32]	validation_0-

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=2, missing=nan, monotone_constraints='()',
             n_estimators=128, n_jobs=8, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='approx', validate_parameters=1, verbosity=None)

In [17]:
submission = model.predict(test_data)
submission = submission.clip(0,20)

In [27]:
out_df = pd.DataFrame({'ID': ID, 'item_cnt_month': submission})
out_df.to_csv('./submission.csv', index=False)